In [3]:
import numpy as np
import keras
import keras.layers as layers
import pickle

In [2]:
glove_mat = np.load("./data/glove_6B_50d.npy")
words = np.load("./data/words_6B_50d.npy")
with open("./data/word_to_ind_6B_50d.pkl", "rb") as f:
    word_to_ind = pickle.load(f)

In [19]:
inputs = layers.Input(shape=(150))

X = layers.Dense(150, activation="relu")(inputs)
X = layers.Dropout(0.2)(X)

X = layers.Dense(100, activation="relu")(X)
X = layers.Dropout(0.2)(X)

Y = layers.Dense(50, activation="relu")(X)

model = keras.Model(inputs=inputs, outputs=Y, name="analogy_model")
model.summary()


Model: "analogy_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 150)]             0         
_________________________________________________________________
dense_20 (Dense)             (None, 150)               22650     
_________________________________________________________________
dropout_13 (Dropout)         (None, 150)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 100)               15100     
_________________________________________________________________
dropout_14 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_22 (Dense)             (None, 50)                5050      
Total params: 42,800
Trainable params: 42,800
Non-trainable params: 0
_________________________________________________

In [32]:
X_train = np.load("./data/google_mikolov/X_train.npy")
X_dev = np.load("./data/google_mikolov/X_dev.npy")
X_test = np.load("./data/google_mikolov/X_test.npy")
X_test_words = np.load("./data/google_mikolov/X_test_words.npy")

Y_train = np.load("./data/google_mikolov/Y_train.npy")
Y_dev = np.load("./data/google_mikolov/Y_dev.npy")
Y_test = np.load("./data/google_mikolov/Y_test.npy")
Y_test_words = np.load("./data/google_mikolov/Y_test_words.npy")

In [29]:
model.compile(loss=keras.losses.MeanSquaredError(), metrics=[keras.metrics.MeanSquaredError()])
model.fit(X_train, Y_train, batch_size=64, epochs=100, validation_data=(X_dev, Y_dev))

Epoch 1/100
245/245 [==============================] - 0s 1ms/step - loss: 0.2680 - mean_squared_error: 0.2680 - val_loss: 0.2517 - val_mean_squared_error: 0.2517
Epoch 2/100
245/245 [==============================] - 0s 998us/step - loss: 0.2673 - mean_squared_error: 0.2673 - val_loss: 0.2511 - val_mean_squared_error: 0.2511
Epoch 3/100
245/245 [==============================] - 0s 1ms/step - loss: 0.2675 - mean_squared_error: 0.2675 - val_loss: 0.2517 - val_mean_squared_error: 0.2517
Epoch 4/100
245/245 [==============================] - 0s 1ms/step - loss: 0.2673 - mean_squared_error: 0.2673 - val_loss: 0.2513 - val_mean_squared_error: 0.2513
Epoch 5/100
245/245 [==============================] - 0s 1ms/step - loss: 0.2674 - mean_squared_error: 0.2674 - val_loss: 0.2515 - val_mean_squared_error: 0.2515
Epoch 6/100
245/245 [==============================] - 0s 1ms/step - loss: 0.2674 - mean_squared_error: 0.2674 - val_loss: 0.2514 - val_mean_squared_error: 0.2514
Epoch 7/100
245/245 

In [30]:
model.evaluate(X_test, Y_test, verbose=2)

62/62 - 0s - loss: 0.2479 - mean_squared_error: 0.2479


[0.24789227545261383, 0.24789227545261383]

In [31]:
Y_predict = model.predict(X_test)
Y_predict.shape

(1955, 50)

In [34]:
def closest_neighbors(embedding, n):
    neighbors = sorted(words, key=lambda word: np.linalg.norm(embedding - glove_mat[word_to_ind[word]]))
    return neighbors[:n]

In [40]:
indices = np.random.randint(Y_predict.shape[0], size=10)
for i, emb in enumerate(Y_predict[indices]):
    print(Y_test_words[indices[i]], closest_neighbors(emb, 5), "Q:", X_test_words[indices[i]])

unfortunate ['fortunately', 'incidentally', 'luckily', 'unfortunately', 'misses'] Q: ['clear' 'unclear' 'fortunate']
reading ['besides', 'shared', 'whereas', 'additionally', 'instance'] Q: ['say' 'saying' 'read']
Gambia ['mo95', 'bdb94', 'k978-1', 'k977-1', 'http://www.mediabynumb'] Q: ['Tokyo' 'Japan' 'Banjul']
Honduras ['http://www.mediabynumb', 'bdb94', 'bb96', 'k977-1', 'mo95'] Q: ['Skopje' 'Macedonia' 'Tegucigalpa']
luckily ['fortunately', 'luckily', 'incidentally', 'srivalo', 'piyanart'] Q: ['rare' 'rarely' 'lucky']
talks ['adding', 'resume', 'sharing', 'discussing', 'discussions'] Q: ['increase' 'increases' 'talk']
California ['serves', 'additionally', 'besides', 'instance', 'addition'] Q: ['Denver' 'Colorado' 'Sacramento']
Mexican ['instance', 'besides', 'exception', 'except', 'likewise'] Q: ['Peru' 'Peruvian' 'Mexico']
safely ['safely', 'fortunately', 'presumably', 'stays', 'ideally'] Q: ['lucky' 'luckily' 'safe']
debugging ['visualizing', 'technobabble', 'optimizes', 'fine-tu